In [2]:
cd "/content/drive/My Drive/Colab Notebooks/matrix_three_road_signs/"

/content/drive/My Drive/Colab Notebooks/matrix_three_road_signs


In [3]:
!pip install hyperopt

In [16]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:


model = Sequential([
  Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),

  Flatten(),          

  Dense(num_classes, activation='softmax')    
    
])

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train)

1088/1088 [==============================] - 5s 5ms/step - loss: 13.5020 - accuracy: 0.7890


In [0]:
def get_cnn_v1(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),

    Flatten(),          

    Dense(num_classes, activation='softmax')    
      
  ])

def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model



In [11]:
model = get_cnn_v1(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 35.7034 - accuracy: 0.7414 - val_loss: 0.2330 - val_accuracy: 0.9429
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2313 - accuracy: 0.9477 - val_loss: 0.1220 - val_accuracy: 0.9710
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1400 - accuracy: 0.9688 - val_loss: 0.1015 - val_accuracy: 0.9777
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.0882 - accuracy: 0.9801 - val_loss: 0.0954 - val_accuracy: 0.9804
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1252 - accuracy: 0.9741 - val_loss: 0.0950 - val_accuracy: 0.9794


In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)



In [0]:
predict(model_trained, X_test, y_test)

0.8390022675736961

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [15]:
def get_cnn_v6(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),          

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')    
      
  ])

train_and_predict( get_cnn_v6(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 6s 23ms/step - loss: 2.5681 - accuracy: 0.4309 - val_loss: 0.4080 - val_accuracy: 0.8967
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 0.5303 - accuracy: 0.8418 - val_loss: 0.1309 - val_accuracy: 0.9726
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2768 - accuracy: 0.9170 - val_loss: 0.0657 - val_accuracy: 0.9818
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1930 - accuracy: 0.9422 - val_loss: 0.0339 - val_accuracy: 0.9924
Epoch 5/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1605 - accuracy: 0.9518 - val_loss: 0.0235 - val_accuracy: 0.9937


0.9664399092970521

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Flatten(),          

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')    
      
  ])

In [27]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]

  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, }

space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4287244682354836, 'dropout_cnn_block_two': 0.4049996167818729, 'dropout_dense_block_one': 0.4336882363688847, 'dropout_dense_block_two': 0.690938463604419}
accuracy=0.8988662362098694
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.31671646195121017, 'dropout_cnn_block_two': 0.4241055331813451, 'dropout_dense_block_one': 0.30128094260744903, 'dropout_dense_block_two': 0.358042472099668}
accuracy=0.9630385637283325
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3908171058773991, 'dropout_cnn_block_two': 0.44419721873522255, 'dropout_dense_block_one': 0.39510857616423256, 'dropout_dense_block_two': 0.5631571637523292}
accuracy=0.9387755393981934
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.35046784067155934, 'dropout_cnn_block_two': 0.3797725258289029, 'dropout_dense_block_one': 0.6121414332857695, 'dropout_dense_block_two': 0.4541528474550765}
accuracy=0.9553288221359253
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.324402725718